## 순환신경망 구현 및 학습

In [35]:
import tensorflow as tf

## 하이퍼 파라미터

In [36]:
EPOCHS = 10
NUM_WORDS = 10000

## 모델 정의

In [37]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.emb = tf.keras.layers.Embedding(NUM_WORDS, 16) # 길이가 NUM_WORDS인 one-hot vector에서 16개의 feature를 뽑아내는것
        #self.rnn = tf.keras.layers.SimpleRNN(32)
        self.rnn = tf.keras.layers.LSTM(32)
        self.dense = tf.keras.layers.Dense(2, activation='softmax')
        # Sentiment Classification : 이번에 다룰 data는 영화에 대한 리뷰를 통해 긍정적인지, 부정적인지를 보기 위해 사용하는 것임.
        # 따라서 길이가 2인 softmax 형태로 되어 있음.
        
        
    def call (self, x, training=None, mask=None):
        x = self.emb(x)
        x = self.rnn(x)
        return self.dense(x)

## 학습, 테스트 루프 정의

In [38]:
# Implement training loop
@tf.function
def train_step(model, inputs, labels, loss_object, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)

    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)

# Implement algorithm test
@tf.function
def test_step(model, images, labels, loss_object, test_loss, test_accuracy):
    predictions = model(images, training=False)

    t_loss = loss_object(labels, predictions)
    test_loss(t_loss)
    test_accuracy(labels, predictions)

## 데이터셋 준비


In [39]:
imdb = tf.keras.datasets.imdb
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words = NUM_WORDS)

# train data와 test data는 길이가 다를 수도 있어서, 최대길이를 정하고 padding도 해줘야함
x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train, value =0, padding = 'pre', maxlen=32)
x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test, value =0, padding = 'pre', maxlen=32)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test,y_test)).batch(32)

## 학습 환경 정의
### 모델 생성, 손실함수, 최적화 알고리즘, 평가지표 정의

In [40]:
# Create model
model = MyModel()

# Define loss and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Define performance metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

## 학습 루프 동작

In [41]:
for epoch in range(EPOCHS):
    for seqs, labels in train_ds:
        train_step(model, seqs, labels, loss_object, optimizer, train_loss, train_accuracy)

    for test_seqs, test_labels in test_ds:
        test_step(model, test_seqs, test_labels, loss_object, test_loss, test_accuracy)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result() * 100,
                          test_loss.result(),
                          test_accuracy.result() * 100))
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

Epoch 1, Loss: 0.5008742809295654, Accuracy: 74.38399505615234, Test Loss: 0.43772271275520325, Test Accuracy: 79.37999725341797
Epoch 2, Loss: 0.36841100454330444, Accuracy: 83.58399963378906, Test Loss: 0.4437551200389862, Test Accuracy: 78.99600219726562
Epoch 3, Loss: 0.3094742000102997, Accuracy: 86.68399810791016, Test Loss: 0.4840843379497528, Test Accuracy: 78.5040054321289
Epoch 4, Loss: 0.24720034003257751, Accuracy: 89.80799865722656, Test Loss: 0.5605773329734802, Test Accuracy: 77.447998046875
Epoch 5, Loss: 0.19194144010543823, Accuracy: 92.3479995727539, Test Loss: 0.6527582406997681, Test Accuracy: 76.30400085449219
Epoch 6, Loss: 0.14359787106513977, Accuracy: 94.45600128173828, Test Loss: 0.76653653383255, Test Accuracy: 76.2040023803711
Epoch 7, Loss: 0.10695689916610718, Accuracy: 95.92400360107422, Test Loss: 0.9472301006317139, Test Accuracy: 75.59600067138672
Epoch 8, Loss: 0.07656870782375336, Accuracy: 97.19999694824219, Test Loss: 1.113505482673645, Test Accur